In [54]:
# 1. Import libraries and load data
import pandas as pd

df = pd.read_csv('bank-additional-full.csv', sep=';')

In [55]:
# Count 'unknown' values in each column BEFORE replacement
unknown_counts = (df == 'unknown').sum()
unknown_counts = unknown_counts[unknown_counts > 0]
print("Columns with 'unknown' values:")
print(unknown_counts)



Columns with 'unknown' values:
job           330
marital        80
education    1731
default      8597
housing       990
loan          990
dtype: int64


## Handling `'unknown'` Values

### Results: Columns with `'unknown'` Values
- **job**: 330  
- **marital**: 80  
- **education**: 1,731  
- **default**: 8,597  
- **housing**: 990  
- **loan**: 990  

---

### Best Practice for This Dataset
Instead of dropping or imputing these values, the recommended approach is to **treat `'unknown'` as its own category**.  

**Why?**
1. **Information Value**  
   - `'unknown'` may carry meaningful signal. For example, a client who refuses to disclose loan or housing status could behave differently from one who openly reports it.  
   - In some cases, “unknown” is not missing at random—it reflects a real client decision or data collection process.

2. **Avoiding Bias**  
   - Imputing with the mode (or another value) risks distorting distributions and introducing bias.  
   - Dropping rows with unknown values would result in losing a **large number of records** (especially for `default` with ~8,600 unknowns).

3. **Consistency with Categorical Encoding**  
   - Since these variables are categorical, adding `"unknown"` as a legitimate category ensures the model treats it appropriately, just like `"yes"` or `"no"`.

---

✅ **Conclusion**:  
For **job, marital, education, default, housing, and loan**, keep `"unknown"` as its own category. This preserves all records and avoids losing potential predictive signal, while staying faithful to the dataset’s real-world context.


In [56]:
#make a new column to indicate if the client has been contacted before
#pdays = number of days since the client was last contacted, 999 means the client has not been contacted before
#so we will create a new column 'prior_contact' where 0 means no prior contact
df['prior_contact'] = df['pdays'].apply(lambda x: 0 if x == 999 else 1)

print("New 'prior_contact' column added to df, indicating if the client has been contacted before.")


New 'prior_contact' column added to df, indicating if the client has been contacted before.


## Feature Engineering: Prior Contact Indicator

The column **`pdays`** represents the number of days since the client was last contacted in a previous campaign.  
- **Special value**: `999` means the client has **not been contacted before**.  
- Any other value means the client has been contacted previously, with the number reflecting days since that contact.

---

### Transformation
We created a new binary column **`prior_contact`**:
- **0** → No prior contact (`pdays = 999`)  
- **1** → Client was contacted before (`pdays != 999`)  

```python
df['prior_contact'] = df['pdays'].apply(lambda x: 0 if x == 999 else 1)


In [57]:
# Descriptive statistics for 'campaign'
print("Descriptive statistics for 'campaign':")
print(df['campaign'].describe())



print(("we see most are between 1 and 3, with a few outliers up to 50."))
# Cap campaign at 6
df['campaign_capped'] = df['campaign'].clip(upper=6)

# Descriptive statistics for 'campaign_capped'
print("\nDescriptive statistics for 'campaign_capped':")
print(df['campaign_capped'].describe())

Descriptive statistics for 'campaign':
count    41188.000000
mean         2.567593
std          2.770014
min          1.000000
25%          1.000000
50%          2.000000
75%          3.000000
max         56.000000
Name: campaign, dtype: float64
we see most are between 1 and 3, with a few outliers up to 50.

Descriptive statistics for 'campaign_capped':
count    41188.000000
mean         2.275274
std          1.550510
min          1.000000
25%          1.000000
50%          2.000000
75%          3.000000
max          6.000000
Name: campaign_capped, dtype: float64


## Descriptive Statistics for `campaign`

The variable **`campaign`** represents the number of contacts performed during this campaign and for this client.  

### Raw `campaign`
- **Count**: 41,188  
- **Mean**: ~2.57 contacts  
- **Std (spread)**: ~2.77  
- **Median (50%)**: 2  
- **25% – 75% (IQR)**: 1 to 3  
- **Max**: 56  

➡️ Interpretation: Most clients were contacted **1–3 times**, but a few extreme outliers (up to 56) inflate the mean and standard deviation. The distribution is highly **right-skewed**.

---

### Capped `campaign_capped` (upper limit = 6)
- **Mean**: ~2.28  
- **Std (spread)**: ~1.55  
- **Median (50%)**: 2  
- **25% – 75% (IQR)**: 1 to 3  
- **Max**: 6  

➡️ Interpretation: By capping at 6, the influence of extreme outliers is reduced.  
- The mean shifts closer to the median (2).  
- The spread becomes tighter (std drops from 2.77 → 1.55).  
- The capped variable better reflects the **typical customer contact range** while avoiding unrealistic noise from rare extreme cases.

---

### Why Capping Helps
- **Exploratory Analysis**: Prevents outliers from skewing averages.  
- **Machine Learning Prep**: Reduces noise; many models are sensitive to extreme values.  
- **Interpretability**: Easier to explain that most customers required ≤6 contacts.

---


In [58]:
#Time to handle data outliers, we will drop the original 'campaign' column
print("\nRemoving the original 'campaign' column.")
df.drop(columns=['campaign'], inplace=True)
print("Original 'campaign' column removed.")
print("New 'campaign_capped' column added with values capped at 6.")



Removing the original 'campaign' column.
Original 'campaign' column removed.
New 'campaign_capped' column added with values capped at 6.


In [59]:
print(df['duration'].describe())

count    41188.000000
mean       258.285010
std        259.279249
min          0.000000
25%        102.000000
50%        180.000000
75%        319.000000
max       4918.000000
Name: duration, dtype: float64


In [60]:
#drop duration column
# If the 'duration' column exists, drop it; otherwise, print a message
#dropped because it is not useful for analysis and can skew results, occurs after the last contact with the client
if 'duration' in df.columns:
	df.drop(columns=['duration'], inplace=True)
	print('dropped duration column from df.')
else:
	print("'duration' column not found in df.")

dropped duration column from df.


## Descriptive Statistics for `duration`

The variable **`duration`** represents the last contact duration (in seconds) with the client.

### Results
- **Count**: 41,188  
- **Mean**: ~258 seconds (~4.3 minutes)  
- **Std (spread)**: ~259 seconds (~4.3 minutes)  
- **Median (50%)**: 180 seconds (3 minutes)  
- **25% – 75% (IQR)**: 102 to 319 seconds (~1.7 to 5.3 minutes)  
- **Min**: 0 seconds (possible no connection or data issue)  
- **Max**: 4,918 seconds (~82 minutes)  

**Interpretation**:  
- Most calls lasted **~2–5 minutes**.  
- The mean is higher than the median, showing a **right-skewed distribution** caused by a small number of very long calls.  
- Some calls have **0-second duration**, likely due to failed connections or placeholders.

---

## Why `duration` Was Dropped

Although highly descriptive, the `duration` variable was **removed from the dataset** for the following reasons:

1. **Data Leakage**  
   - `duration` is only known **after** the last contact is completed.  
   - Including it in a predictive model would give the model access to “future” information unavailable at the time of prediction.  
   - This would artificially inflate accuracy and produce misleading results.

2. **Skew and Outliers**  
   - The presence of extremely long calls (up to 82 minutes) makes the variable highly skewed, which can distort statistical analysis and model training.

3. **Actionability**  
   - Business stakeholders cannot know the call duration **before the call is made**, making it less useful for real-world targeting decisions.  

---

✅ **Conclusion**:  
`duration` is informative for describing past campaign behavior, but it is **not valid for predictive modeling**. Dropping it ensures that the analysis focuses only on features available **before or during** the campaign, preventing data leakage and improving real-world applicability.


In [61]:
#make a new column to indicate if the client has been contacted before
#pdays = number of days since the client was last contacted, 999 means the client has not been contacted before
#so we will create a new column 'prior_contact' where 0 means no prior contact
df['prior_contact'] = df['pdays'].apply(lambda x: 0 if x == 999 else 1)

print("New 'prior_contact' column added to df, indicating if the client has been contacted before.")


New 'prior_contact' column added to df, indicating if the client has been contacted before.


Creating a Prior Contact Indicator

In the dataset, the column 'pdays' represents the number of days since the client
was last contacted.

- If pdays = 999 → the client has never been contacted before.
- Any other value → the client was contacted previously.

This raw encoding (999 as a placeholder) is not intuitive, so we create a new
binary feature: 'prior_contact'.

What I Did:
------------
I created a binary column:
    0 → No prior contact (pdays = 999)
    1 → Client was contacted before (pdays ≠ 999)

Why I Did It:
--------------
1. Clarity: avoids relying on the special-case code '999'.
2. Model Readiness: binary indicators are more suitable for ML models.
3. Interpretability: easier for humans to understand at a glance.



In [62]:
#lets drop the 'pdays' column
#we will keep the 'prior_contact' column    
df.drop(columns=['pdays'], inplace=True)
print("Dropped 'pdays' column from df, keeping 'prior_contact' column to indicate prior contact status.")

Dropped 'pdays' column from df, keeping 'prior_contact' column to indicate prior contact status.


In [63]:
#put target variable 'y' into binary format
#map 'yes' to 1 and 'no' to 0
df['y'] = df['y'].map({'yes': 1, 'no': 0}).astype(int)

## One Hot Encoding Notes

One-hot encoding is a data preprocessing step used to convert categorical variables (like "job", "education", "month", etc.) into a format that machine learning algorithms can understand.

Most machine learning models (especially logistic regression, decision trees, etc.) can't work directly with text labels — they need numbers



## Why Use In My Code?
One-hot encoding creates a new column for each unique value in a categorical column and assigns a 1 or 0 to indicate presence.

This avoids introducing false ordinal relationships (like if you encoded them as 0, 1, 2), which would wrongly suggest that "technician" > "admin."

I'm building a model to predict whether someone will subscribe to a term deposit (a classification task).

Many of the features are categorical:

['job', 'marital', 'education', 'default', 'housing', 'loan',
 'contact', 'month', 'day_of_week', 'poutcome']

 
 To make these usable in My logistic regression or other ML model, I must convert them to numbers — and one-hot encoding is the standard, safe way to do it.


In [64]:
# Define the original categorical columns
categorical_cols = [
    'job', 'marital', 'education', 'default', 'housing', 'loan',
    'contact', 'month', 'day_of_week', 'poutcome'
]

# Make a copy of df BEFORE encoding for comparison later
df_beforeEncoding = df.copy()

# Convert categorical variables to dummy/indicator variables
df_afterEncoding = pd.get_dummies(df, columns=categorical_cols, drop_first=True)

# Informative messages
print("All categorical variables in df have been converted to dummy variables.")
print("Converted categorical columns to dummy variables:")
print(categorical_cols)
print()

# Compare column sets to find new dummy variables created
new_columns = set(df_afterEncoding.columns) - set(df_beforeEncoding.columns)
print("New columns created by one-hot encoding:")
print(sorted(new_columns))  # sorted to make it easier to read

All categorical variables in df have been converted to dummy variables.
Converted categorical columns to dummy variables:
['job', 'marital', 'education', 'default', 'housing', 'loan', 'contact', 'month', 'day_of_week', 'poutcome']

New columns created by one-hot encoding:
['contact_telephone', 'day_of_week_mon', 'day_of_week_thu', 'day_of_week_tue', 'day_of_week_wed', 'default_unknown', 'default_yes', 'education_basic.6y', 'education_basic.9y', 'education_high.school', 'education_illiterate', 'education_professional.course', 'education_university.degree', 'education_unknown', 'housing_unknown', 'housing_yes', 'job_blue-collar', 'job_entrepreneur', 'job_housemaid', 'job_management', 'job_retired', 'job_self-employed', 'job_services', 'job_student', 'job_technician', 'job_unemployed', 'job_unknown', 'loan_unknown', 'loan_yes', 'marital_married', 'marital_single', 'marital_unknown', 'month_aug', 'month_dec', 'month_jul', 'month_jun', 'month_mar', 'month_may', 'month_nov', 'month_oct', 'mo

## 📊 DataFrames Overview

### 1. `df_beforeEncoding`
This is the **original** DataFrame, with some basic cleaning applied. It includes the following **categorical columns** that require encoding:

```python
['job', 'marital', 'education', 'default', 'housing', 'loan', 
 'contact', 'month', 'day_of_week', 'poutcome']



## 📊 DataFrames Overview

### 2. `df_afterEncoding`
This is the **updated** DataFrame, with One Hot Encoding applied. It includes the following **Numerical columns** that have been encoded:

```python
['contact_telephone', 'day_of_week_mon', 'day_of_week_thu', 'day_of_week_tue', 'day_of_week_wed', 'default_unknown', 'default_yes', 'education_basic.6y', 'education_basic.9y', 'education_high.school', 'education_illiterate', 'education_professional.course', 'education_university.degree', 'education_unknown', 'housing_unknown', 'housing_yes', 'job_blue-collar', 'job_entrepreneur', 'job_housemaid', 'job_management', 'job_retired', 'job_self-employed', 'job_services', 'job_student', 'job_technician', 'job_unemployed', 'job_unknown', 'loan_unknown', 'loan_yes', 'marital_married', 'marital_single', 'marital_unknown', 'month_aug', 'month_dec', 'month_jul', 'month_jun', 'month_mar', 'month_may', 'month_nov', 'month_oct', 'month_sep', 'poutcome_nonexistent', 'poutcome_success']


In [65]:
#Check for duplicate values, columns, data redundancy


print(f"Shape before dropping duplicates: {df_afterEncoding.shape}")
with_duplicates = df_afterEncoding.columns[df_afterEncoding.columns.duplicated()].tolist()

if with_duplicates:
    print("\nDuplicate columns found after encoding:")
    print(with_duplicates)
else:
    print("\nNo duplicate columns found after encoding.")

duplicate_values = df_afterEncoding.duplicated().sum()


print(f"\nNumber of duplicate rows in df_afterEncoding: {duplicate_values}")
if duplicate_values > 0:
    print("Consider removing duplicate rows to ensure data integrity.")
    df_afterEncoding.drop_duplicates(inplace=True)
else:
    print("No duplicate rows found in df_afterEncoding.")

print()
print(f"Shape after dropping duplicates: {df_afterEncoding.shape}")

Shape before dropping duplicates: (41188, 53)

No duplicate columns found after encoding.

Number of duplicate rows in df_afterEncoding: 1860
Consider removing duplicate rows to ensure data integrity.

Shape after dropping duplicates: (39328, 53)


In [66]:
df_afterEncoding.to_csv('cleaned_bank.csv', index=False)
print("Cleaned data saved to 'cleaned_bank.csv'.")

Cleaned data saved to 'cleaned_bank.csv'.
